https://towardsdatascience.com/graphs-and-paths-pagerank-54f180a1aa0a

In [2]:
import findspark
findspark.init()
import pyspark

In [3]:
sc = pyspark.SparkContext(appName="PageRank")

In [14]:
def computeContribe(urls, rank):
    """calculates URL contributions to the rank of others URLs"""
    num_urls = len(urls)
    contribs = []
    for url in urls:
        contribs.append((url, rank/num_urls))
    return contribs

In [ ]:
graph = sc.textFile("C:/Users/EASYFRONT/Desktop/bigdata/seance02/pagerank/web-Google.txt")
#clean up graph rdd from comment and parse it
graph_cup = graph.filter(lambda x : "#" not in x).map(lambda x : x.split("\t"))
graph_cup.collect()

In [ ]:
graph_cup.getNumPartitions()

In [ ]:
#draft algorithm
links = graph_cup.groupByKey().partitionBy(100).cache()
#initialize the ranks to 1 for each URL
ranks = links.map(lambda url_neighbors : (url_neighbors[0],1.0) )
#ranks.collect()
for iteration in range(2):
    contribs = links.join(ranks).flatMap(lambda url_urls_rank : computeContribe(url_urls_rank[1][0],url_urls_rank[1][0]))
    ranks = contribs.reduceByKey(lambda x,y : x+y).mapValues(lambda rank : rank*0.85 + 0.15)

for (link,rank) in ranks.sortBy(lambda x:-x[1]).take(10):
    print("%s has rank : %s." % (link, rank))

In [ ]:
sc.stop()

SMALL GRAPH EXAMPLE

In [4]:
rdd_medium_example = sc.parallelize([("Amzn","Twtr"),("Amzn","Medm"),("Fb","Amzn"),("Fb","Twtr"),("Medm","Twtr"),("Medm","Amzn"),("Medm","Mspc"),("Medm","Fb"),("Twtr","Medm"),("Mspc","Twtr")])
rdd_medium_example.collect()

[('Amzn', 'Twtr'),
 ('Amzn', 'Medm'),
 ('Fb', 'Amzn'),
 ('Fb', 'Twtr'),
 ('Medm', 'Twtr'),
 ('Medm', 'Amzn'),
 ('Medm', 'Mspc'),
 ('Medm', 'Fb'),
 ('Twtr', 'Medm'),
 ('Mspc', 'Twtr')]

In [8]:
nodes = rdd_medium_example.flatMap(lambda x : [x[0]]+[x[1]]).distinct()
nbnodes = nodes.count()
nbnodes

5

In [11]:
#initialization phase
ranks = nodes.map(lambda x : (x,1/nbnodes))
ranks.collect()

[('Amzn', 0.2), ('Twtr', 0.2), ('Medm', 0.2), ('Fb', 0.2), ('Mspc', 0.2)]

In [ ]:
#compute following structure (node, all vertices that comme out of from node (aretes sortantes))
links = rdd_medium_example.groupByKey()
for iteration in range(2):
    contribs = links.join(ranks).flatMap(lambda url_urls_rank : computeContribe(url_urls_rank[1][0],url_urls_rank[1][1]))
    ranks = contribs.reduceByKey(lambda x,y : x+y).mapValues(lambda rank : round(rank*0.85 + 0.15/nbnodes,2))

for (link,rank) in ranks.sortBy(lambda x:-x[1]).collect():
    print("%s has rank : %s." % (link, rank))

DEMISTIFYING ONE ITERATION

In [9]:
links = rdd_medium_example.groupByKey()
links = links.map(lambda x : (x[0], list(x[1])) ) 
links.collect()

[('Amzn', ['Twtr', 'Medm']),
 ('Medm', ['Twtr', 'Amzn', 'Mspc', 'Fb']),
 ('Twtr', ['Medm']),
 ('Fb', ['Amzn', 'Twtr']),
 ('Mspc', ['Twtr'])]

In [12]:
links_ranks = links.join(ranks)
links_ranks.collect()

[('Medm', (['Twtr', 'Amzn', 'Mspc', 'Fb'], 0.2)),
 ('Twtr', (['Medm'], 0.2)),
 ('Amzn', (['Twtr', 'Medm'], 0.2)),
 ('Fb', (['Amzn', 'Twtr'], 0.2)),
 ('Mspc', (['Twtr'], 0.2))]

In [15]:
contribs = links_ranks.flatMap(lambda url_urls_rank : computeContribe(url_urls_rank[1][0],url_urls_rank[1][1]))
contribs.collect()

[('Twtr', 0.05),
 ('Amzn', 0.05),
 ('Mspc', 0.05),
 ('Fb', 0.05),
 ('Medm', 0.2),
 ('Twtr', 0.1),
 ('Medm', 0.1),
 ('Amzn', 0.1),
 ('Twtr', 0.1),
 ('Twtr', 0.2)]

In [16]:
ranks_it1 = contribs.reduceByKey(lambda x,y : x+y).mapValues(lambda rank : rank*0.85 + 0.15/nbnodes)
ranks_it1.collect()

[('Twtr', 0.4125000000000001),
 ('Medm', 0.28500000000000003),
 ('Amzn', 0.1575),
 ('Mspc', 0.07250000000000001),
 ('Fb', 0.07250000000000001)]

In [ ]:
links2 = rdd_medium_example.groupByKey()
links2 = links2.map(lambda x : (x[0], list(x[1])) ) 
links2 = links2.mapValues(lambda x : 1)
links2.collect()